# Regulator: Market Competition & Collusion Detection Demo

This notebook demonstrates the core functionality of the Regulator system for simulating market competition and detecting collusive behavior.

## What is Regulator?

Regulator is a comprehensive system for:

1. **Market Simulation**: Oligopolistic price competition with dynamic demand elasticity
2. **Agent Framework**: Multiple firm types (random, tit-for-tat, adaptive learning, chat-enabled)
3. **Detection Systems**: ML-based and LLM-powered collusion detection
4. **Regulatory Monitoring**: Enhanced regulator with graduated penalties and risk scoring

**Key Features:**
- Real-time collusion detection using machine learning
- Natural language chat monitoring with LLM analysis
- Economic validation and consistency checks
- Interactive dashboard for visualization

## Setup and Installation

In [ ]:
! [ ! -d "regulator" ] && git clone https://github.com/bangyen/regulator.git
! cd regulator && pip install -e .

print("Setup complete!")

## Imports and Configuration

In [ ]:
import os

os.chdir("./regulator")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.cartel.cartel_env import CartelEnv
from src.agents.firm_agents import RandomAgent, TitForTatAgent, BestResponseAgent
from src.agents.enhanced_regulator import EnhancedRegulator
from src.agents.ml_regulator import MLRegulator

print("Imports successful!")

## Market Simulation with Collusion Detection

In [ ]:
def run_market_simulation():
    """Run a simple market simulation with collusion detection."""
    print("Regulator Demo - Market Simulation with Collusion Detection")
    print("=" * 60)
    
    # Create market environment with correct parameters
    env = CartelEnv(
        n_firms=3,
        max_steps=50,
        price_elasticity=-1.5,  # Price elasticity of demand (negative value)
        use_capacity_constraints=True,
        capacity=[100, 100, 100],  # Capacity for each firm
        marginal_cost=10.0,
        demand_intercept=100.0,
        demand_slope=-1.0
    )
    
    # Create different types of agents
    agents = [
        RandomAgent(agent_id=0),
        TitForTatAgent(agent_id=1),
        BestResponseAgent(agent_id=2)
    ]
    
    # Create regulators with more sensitive thresholds to see some detection
    enhanced_regulator = EnhancedRegulator(
        use_graduated_penalties=True,
        use_market_awareness=True,
        parallel_threshold=2.0,  # More sensitive
        structural_break_threshold=10.0  # More sensitive
    )
    
    ml_regulator = MLRegulator(
        use_ml_detection=True,
        ml_anomaly_threshold=0.05,  # More sensitive
        ml_collusion_threshold=0.5  # More sensitive
    )
    
    # Run simulation
    obs, info = env.reset()  # Reset returns (observation, info) tuple
    prices_history = []
    profits_history = []
    violations_history = []
    
    print("Running simulation...")
    
    for step in range(env.max_steps):
        # Get actions from agents using choose_price method
        actions = []
        for i, agent in enumerate(agents):
            action = agent.choose_price(obs, env)  # Pass full observation to each agent
            actions.append(action)
        
        # Step environment - returns 5 values in Gymnasium format
        obs, rewards, terminated, truncated, info = env.step(actions)
        
        # Extract prices and profits from info dictionary
        prices = info['prices'].tolist()  # Convert numpy array to list
        profits = info['total_profits'].tolist()  # Convert numpy array to list
        
        prices_history.append(prices.copy())
        profits_history.append(profits.copy())
        
        # Update agent histories so they can learn from previous prices
        for i, agent in enumerate(agents):
            # Get rival prices (all other firms' prices)
            rival_prices = np.array([prices[j] for j in range(len(agents)) if j != i])
            agent.update_history(prices[i], rival_prices)
        
        # Monitor with regulators - convert back to numpy arrays for regulator methods
        prices_array = np.array(prices)
        enhanced_result = enhanced_regulator.monitor_step(prices_array, step, info)
        ml_result = ml_regulator.monitor_step(prices_array, step, info)
        
        # Extract more detailed information for visualization
        violations = {
            'enhanced_violations': enhanced_result.get('violation_details', []),
            'ml_violations': ml_result.get('violation_details', []),
            'market_volatility': enhanced_result.get('market_volatility', 0.0),
            'ml_anomaly_score': ml_result.get('ml_anomaly_score', 0.0),
            'ml_collusion_probability': ml_result.get('ml_collusion_probability', 0.0),
            'fines_applied': enhanced_result.get('fines_applied', [0, 0, 0])
        }
        violations_history.append(violations)
        
        if step % 10 == 0:
            print(
                f"Step {step}:\n"
                f"  Prices  = {[f'{p:.2f}' for p in prices]}\n"
                f"  Profits = {[f'{pr:.2f}' for pr in profits]}\n"
                f"  Market volatility: {violations['market_volatility']:.3f}"
            )
    
    return prices_history, profits_history, violations_history

# Run the simulation
prices_history, profits_history, violations_history = run_market_simulation()

## Visualization and Analysis

In [ ]:
# Visualize price trajectories and market volatility
plt.figure(figsize=(12, 5))

# Plot prices over time
plt.subplot(1, 2, 1)
for i in range(3):
    prices = [step[i] for step in prices_history]
    plt.plot(prices, label=f'Firm {i}', marker='o', markersize=3)
plt.title('Price Trajectories')
plt.xlabel('Step')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot market volatility over time
plt.subplot(1, 2, 2)
market_volatility = [step['market_volatility'] for step in violations_history]
plt.plot(market_volatility, label='Market Volatility', marker='^', markersize=3, color='red')
plt.title('Market Volatility Over Time')
plt.xlabel('Step')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\nSimulation Summary:")
print("=" * 40)
print(f"Total steps: {len(prices_history)}")
print(f"Average prices: {[round(np.mean([step[i] for step in prices_history]), 2) for i in range(3)]}")
print(f"Average profits: {[round(np.mean([step[i] for step in profits_history]), 2) for i in range(3)]}")
print(f"Max market volatility: {max([step['market_volatility'] for step in violations_history]):.3f}")
print(f"Max ML anomaly score: {max([step['ml_anomaly_score'] for step in violations_history]):.3f}")
print(f"Max ML collusion probability: {max([step['ml_collusion_probability'] for step in violations_history]):.3f}")
print(f"Total fines applied: {sum([sum(step['fines_applied']) for step in violations_history]):.2f}")
print(f"Steps with fines: {sum(1 for step in violations_history if sum(step['fines_applied']) > 0)}")

## Key Takeaways

This demo shows how the Regulator system works:

1. **Market Simulation**: Creates realistic oligopolistic competition with dynamic pricing
2. **Agent Diversity**: Different firm types (Random, Tit-for-Tat, Best Response) create varied market dynamics
3. **Dual Detection**: Enhanced regulator with graduated penalties and ML-based anomaly detection
4. **Real-time Monitoring**: Continuous risk scoring and violation detection throughout the simulation

### Next Steps

- **Run experiments**: `regulator experiment --steps 100 --firms "random,tit_for_tat,best_response"`
- **Launch dashboard**: `regulator dashboard` for interactive visualization
- **Try chat agents**: Use `CollusiveChatAgent` and `CompetitiveChatAgent` with LLM detection
- **Leniency programs**: Run `python scripts/leniency_experiment.py` to test whistleblower dynamics
- **Custom agents**: Create new agent types in `src/agents/firm_agents.py`

### Advanced Features

- **Economic validation**: Built-in consistency checks for market dynamics
- **ML detection**: LightGBM-based collusion classification
- **LLM monitoring**: OpenAI-powered natural language analysis
- **Graduated penalties**: Dynamic penalty structures based on violation severity

For more information, visit the [GitHub repository](https://github.com/bangyen/regulator).
